In [2]:
################################################
# # Spark Python Home ***

# 	$ vi $SPARK_HOME/conf/spark-env.sh
	
# # CONDA & PYTHON
# export CONDA_HOME=/opt/miniconda3/bin
# export PYTHON_HOME=${CONDA_HOME}/python
# export PYSPARK_PYTHON=${PYTHON_HOME}
# export PYSPARK_DRIVER_PYTHON=${PYTHON_HOME}

# export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
# export SPARK_DIST_CLASSPATH=$(/usr/local/hadoop/bin/hadoop classpath)
# export LD_LIBRARY_PATH=$JAVA_LIBRARY_PATH
# export SPARK_CLASSPATH=$SPARK_CLASSPATH:/usr/local/spark/jars/*

# export SPARK_MASTER_IP=localhost
# export SPARK_MASTER_PORT=7077
# export SPARK_WORKER_MEMORY=4g
#
################################################
# # .bashrc

#  $ vi ~/.bashrc
#
# JAVA ENV SETTINGS

# # export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64
# export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


# export JRE_HOME=${JAVA_HOME}/jre
# export CLASSPATH=.:${JAVA_HOME}/lib:${JRE_HOME}/lib
# export MYSQL_LIB=${JRE_HOME}/lib/ext/mysql
# export PATH=${JAVA_HOME}/bin:$PATH

# # HADOOP ENV SETTINGS
# export HADOOP_HOME=/usr/local/hadoop
# export HADOOP_COMMON_LIB_NATIVE_DIR=$HADOOP_HOME/lib/native
# export JAVA_LIBRARY_PATH==${HADOOP_HOME}/lib/native
# export PATH=${HADOOP_HOME}/bin:${HADOOP_HOME}/sbin:$PATH

# # SPARK EVN SETTINGS
# export SPARK_HOME=/usr/local/spark
# export PATH=${SPARK_HOME}/bin:$PATH
# # Conflict with Hadoop: start-all.sh
# # export PATH=${SPARK_HOME}/sbin:$PATH

# # CONDA & PYTHON
# export CONDA_HOME=/opt/miniconda3/bin
# export PYTHON_HOME=${CONDA_HOME}/python
# export PYSPARK_PYTHON=${PYTHON_HOME}
# export PYSPARK_DRIVER_PYTHON=${PYTHON_HOME}

# export PYSPARK_SUBMIT_ARGS="--master local[2] pyspark-shell"
#
# # source ~/.bashrc
################################################


In [9]:
# Set the PySpark environment variables
# FOR 
import os
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[2] pyspark-shell"
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages mysql:mysql-connector-j-9.0.0.jar pyspark-shell'

In [4]:
import findspark

findspark.init()  # 初始化Spark

# $ ./bin/spark-shell --driver-class-path postgresql-9.4.1207.jar 
#                     --jars postgresql-9.4.1207.jar

In [5]:

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MySQL Connection") \
    .config("spark.jars", "/usr/local/spark/jars/mysql-connector-j-9.0.0.jar") \
    .getOrCreate()

# url = "jdbc:mysql://localhost:3306/spark"
url = "jdbc:mysql://localhost/spark"
# URL=jdbc:mysql://[host][:port]/[database]
# mysql> SHOW VARIABLES WHERE Variable_name = 'hostname';
properties = {
    "user": "root",
    "password": "Smart_Battery@2022"
}

df = spark.read.jdbc(url=url, table="student", properties=properties)
df.show()

+---+--------------------+------+---+
| id|                name|gender|age|
+---+--------------------+------+---+
|  1|Xueqian             |  F   | 23|
|  2|Weiliang            |  M   | 24|
|  3|Niro                |  F   | 23|
+---+--------------------+------+---+



In [7]:
from pyspark.sql.types import Row
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

studentRDD = spark.sparkContext.parallelize(["3 Rongcheng M 26","4 Guanhua M 27"]).map(lambda line : line.split(" "))

## 设置模式信息
schema = StructType([StructField("name", StringType(), True),StructField("gender", StringType(), True),StructField("age",IntegerType(), True)])
rowRDD = studentRDD.map(lambda p : Row(p[1].strip(), p[2].strip(),int(p[3])))


## 建立起Row对象和模式之间的对应关系，也就是把数据和模式对应起来

studentDF = spark.createDataFrame(rowRDD, schema)
prop = {}
prop['user'] = 'root'
prop['password'] = 'Smart_Battery@2022'
prop['driver'] = "com.mysql.jdbc.Driver"
studentDF.write.jdbc("jdbc:mysql://localhost:3306/spark",'student','append', prop)

In [8]:
df.show()

+---+--------------------+------+---+
| id|                name|gender|age|
+---+--------------------+------+---+
|  1|Xueqian             |  F   | 23|
|  2|Weiliang            |  M   | 24|
|  3|Niro                |  F   | 23|
|  4|Guanhua             |  M   | 27|
|  5|Rongcheng           |  M   | 26|
+---+--------------------+------+---+



In [ ]:
# Stop the SparkSession
spark.stop()